In [16]:
%%capture --no-stderr
%pip install --quiet -U langchain_core langgraph langchain_openai

In [2]:
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 配置API设置
api_key = os.environ["DEEPSEEK_API_KEY"]
api_base = os.environ["OPENAI_API_BASE"]

# 在创建ChatOpenAI实例时使用这些设置
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="deepseek-chat",  # DeepSeek的模型名称
    openai_api_key=api_key,
    openai_api_base=api_base,
    temperature=0.7
)

# 打印API设置
print(api_key)
print(api_base)

sk-973abc04ec464088aabcd5a5dfcfe3b0
https://api.deepseek.com/v1


In [3]:
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content='Hi Bob! 👋 Nice to meet you—I’m DeepSeek Chat. How can I help you today? 😊', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 8, 'total_tokens': 35, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}, 'prompt_cache_hit_tokens': 0, 'prompt_cache_miss_tokens': 8}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_3d5141a69a_prod0225', 'id': '5ce60ea3-d6a2-40b1-8c0e-b54d77f5d6e2', 'finish_reason': 'stop', 'logprobs': None}, id='run-94aef685-3036-491e-b8a9-cf365f2fd793-0', usage_metadata={'input_tokens': 8, 'output_tokens': 27, 'total_tokens': 35, 'input_token_details': {'cache_read': 0}, 'output_token_details': {}})

In [24]:
from langgraph.graph import MessagesState
#MessagesState内有一个messages字段，用于存储消息，messages是一个列表，列表中每个元素是一个Message对象
class State(MessagesState):
    #state继承MessagesState
    #state中有一个summary字段，用于存储摘要
    summary: str

In [25]:
# Define the logic to call the model
def call_model(state: State):
    
    # Get summary if it exists
    summary = state.get("summary", "")

    # If there is summary, then we add it
    if summary:
        
        # Add summary to system message
        system_message = f"Summary of conversation earlier: {summary}"
        
        # Append summary to any newer messages
        messages = [SystemMessage(content=system_message)] + state["messages"]
    
    else:
        messages = state["messages"]
    
    response = model.invoke(messages)
    return {"messages": response}

In [26]:
def summarize_conversation(state: State):
    
    # First, we get any existing summary
    summary = state.get("summary", "")

    # Create our summarization prompt 
    if summary:
        
        # A summary already exists
        summary_message = (
            f"This is summary of the conversation to date: {summary}\n\n"
            "Extend the summary by taking into account the new messages above:"
        )
        
    else:
        summary_message = "Create a summary of the conversation above:"

    #上面是创建一个总结的prompt，然后将prompt添加到messages中
    # Add prompt to our history
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    
    # Delete all but the 2 most recent messages
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

In [27]:
from langgraph.graph import END
# Determine whether to end or summarize the conversation
def should_continue(state: State):
    
    """Return the next node to execute."""
    
    messages = state["messages"]
    
    # If there are more than six messages, then we summarize the conversation
    if len(messages) > 6:
        return "summarize_conversation"
    
    # Otherwise we can just end
    return END

In [28]:
from IPython.display import Image, display
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START

# Define a new graph
workflow = StateGraph(State)
workflow.add_node("conversation", call_model)
workflow.add_node(summarize_conversation)

# Set the entrypoint as conversation
workflow.add_edge(START, "conversation")
workflow.add_conditional_edges("conversation", should_continue)
workflow.add_edge("summarize_conversation", END)

# Compile
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)

In [29]:
# Create a thread
config = {"configurable": {"thread_id": "1"}}

# Start conversation
input_message = HumanMessage(content="hi! I'm Lance")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="what's my name?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

input_message = HumanMessage(content="i like the 49ers!")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

Hi Lance! 👋 Nice to meet you—what’s on your mind today? Whether it’s questions, random thoughts, or just saying hello, I’m here for it. 😊  

(Also, love the name—are you a fan of knights, spears, or perhaps a certain Pokémon trainer? �⚔️🐉)
================================== Ai Message ==================================

Your name is **Lance**! Unless you’ve secretly switched it in the last few minutes—then you’ll have to fill me in. 😉  

(Still holding out hope you’re either a knight or a dragon tamer, though. 🏰🐲)
================================== Ai Message ==================================

Nice! The **49ers** are a legendary franchise—great choice! 🏈💛❤️  

Are you hyped for this season? Any favorite players (past or present)? Deebo’s explosiveness? Kittle’s energy? Or maybe you’re still riding the high of the Montana/Young era?  

Also, how do you feel about the NFC West rivalry drama? (Looking at yo

In [30]:
graph.get_state(config).values.get("summary","")

''

In [31]:
input_message = HumanMessage(content="i like Nick Bosa, isn't he the highest paid defensive player?")
output = graph.invoke({"messages": [input_message]}, config) 
for m in output['messages'][-1:]:
    m.pretty_print()

================================== Ai Message ==================================

You're absolutely right! **Nick Bosa** is a *monster* on the field and now officially the **highest-paid defensive player in NFL history**! 🏈💰  

After his 2022 **DPOY** season and holding out for a mega-deal, the 49ers locked him down in 2023 with a **5-year, $170 million contract** ($34M/year), including **$122.5 million guaranteed**. That dethroned Aaron Donald’s previous record.  

**Why he’s worth it:**  
- Elite pass rusher (18.5 sacks in ‘22, led the league)  
- Disrupts offenses *every* snap (double-team magnet)  
- Clutch in big games (see: 2023 playoffs)  

The only question now: Can he and that stacked D-line bring the Niners a **Super Bowl** this season? 🔥  

*(Also, how badly do you miss his brother Joey in the Chargers games? The Bosa family genes are unfair. 😂)*


In [32]:
graph.get_state(config).values.get("summary","")

"### **Summary of Our Conversation:**  \n\n1. **Introduction:** You introduced yourself as **Lance**, and I playfully guessed if your name was inspired by knights, spears, or Pokémon (Lance the Dragon Trainer).  \n\n2. **49ers Fandom:** You mentioned loving the **San Francisco 49ers**, and we discussed:  \n   - Your favorite player, **Nick Bosa**, and his record-breaking **$170M contract** (highest-paid defensive player ever).  \n   - His dominance (18.5 sacks in 2022, DPOY, elite pass-rushing).  \n   - Hopes for a **Super Bowl run** with the 49ers' stacked defense.  \n   - Lighthearted jokes about the **Bosa family genes** (shoutout to Joey Bosa).  \n\n3. **NFC West Rivalries:** Briefly touched on the **Seahawks and Rams drama**.  \n\n**Theme:** A mix of fun banter, football hype, and appreciation for Nick Bosa’s greatness! 🏈💪  \n\nLet me know if you want to dive deeper into anything!"